Calculando a matriz de pesos ótima usando bootstraping

1. Ler os dados do projeto dentro de Julia
2. Calcular os momentos usando os dados
3. Calcular os momentos várias vezes usando retiradas aleatórias com recomposição, para ver o quanto os momentos variam

In [1]:
#lendo dados do projeto: trilemma_data.xlsx
# using Pkg
# Pkg.add("XLSX")

using XLSX

In [2]:
using DataFrames

In [3]:
using Statistics
using Gadfly

In [4]:
ENV["COLUMNS"] = 240 #para aumentar o número de colunas mostradas

240

In [5]:
df = DataFrame(XLSX.readtable("trilemma_data.xlsx", "Sheet1")...)

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0000547-55.2014.8.26.0291,2014-01-22,2014-01-29,2015-09-16,2018-07-12,0.233333,19.8333,34.3333,54.4,1.0,3.49201e6,1.38557e6,3.656e6,0.442533,0.941905,0.442533,2.53837e7,2.36164e7
2,0000731-65.2011.8.26.0695,2011-03-22,2011-04-08,2011-10-20,2017-04-20,0.566667,6.5,66.9667,74.0333,1.0,1.58332e6,9.64064e5,1.61192e5,0.81201,0.941905,0.81201,2.77487e6,1.0668e6
3,0003124-79.2014.8.26.0299,2014-05-28,2014-09-25,2016-07-18,2018-07-02,4.0,22.0667,23.8,49.8667,1.0,9.85081e6,1.63535e5,3.30049e6,0.36744,0.968244,0.36744,1.1296e7,3.32642e6
4,0006202-38.2010.8.26.0100,2010-02-12,2010-03-29,2011-01-03,2017-08-25,1.5,9.33333,80.8667,91.7,1.0,3.58351e7,1.72202e6,4.18984e5,0.535184,0.941905,0.535184,1.65361e7,1.15454e7
5,0011100-26.2012.8.26.0100,2012-03-06,2012-04-04,2014-05-15,2015-11-24,0.966667,25.7,18.6,45.2667,1.0,1.43452e7,0.0,1.41472e7,0.465931,0.0,0.465931,1.86502e7,5.63179e6
6,0014297-52.2013.8.26.0100,2013-02-07,2013-02-22,2014-07-21,2018-05-15,0.5,17.1333,46.4667,64.1,1.0,3.1438e7,1.41865e6,1.48989e7,0.396523,0.941905,0.536727,6.79508e7,1.23996e7
7,0016095-59.2013.8.26.0161,2013-06-17,2014-03-18,2016-06-06,2018-11-01,9.13333,27.0333,29.2667,65.4333,1.0,1.00002e7,75754.0,1.02842e5,0.814917,0.941905,0.814917,3.684e6,259066.0
8,0023189-81.2012.8.26.0100,2012-05-07,2012-06-22,2013-06-07,2019-01-15,1.53333,11.6667,68.2667,81.4667,1.0,4.86213e6,0.0,3.844e6,0.206332,0.0,0.206332,2.43727e6,755068.0
9,0025113-93.2013.8.26.0100,2013-04-03,2013-08-16,2014-09-17,2018-09-24,4.5,13.2333,48.9333,66.6667,1.0,1.07158e6,0.0,150000.0,0.448017,0.0,0.448017,299845.0,243594.0


In [6]:
names(df)

32-element Array{Symbol,1}:
 :processo
 :fil_date
 :data_start
 :plan_date
 :final_date
 :months_fil_start
 :months_start_plan
 :months_plan_final
 :months_fil_final
 :in_court
 :D_junior
 :D_labor
 :D_senior
 ⋮
 :Vh
 :L
 :D
 :Dj_D
 :L_D
 :Vh_D
 :Reorg_incourt
 :Reorg_precourt
 :Liq_incourt
 :Liq_precourt
 :tempo_medio_agc
 :ln_n_months

In [7]:
INCOURT = filter(df -> df.in_court .== 1.0, df);
PRECOURT = filter(df -> df.in_court .== 0.0, df);

MM1: average log number of months between observed proposals, for incourt cases

In [8]:
mm1 = mean(skipmissing(df.ln_n_months))

2.1118291318473066

In [9]:
#bootstrap momento 1

#B é o número de simulações no bootstrap

using Random

Random.seed!(2021);

In [10]:
for i in 1:10
    println(rand(5))
end

[0.4057963453000579, 0.06577381398805326, 0.398161853516253, 0.16381573194486343, 0.7830936283777754]
[0.13411536438779015, 0.8831208932541377, 0.3868745381902321, 0.2421045713808192, 0.13158810652098185]
[0.08533100886462575, 0.3300990804142949, 0.654601431594249, 0.46732767220577864, 0.88933420950256]
[0.7052608184890443, 0.09003163507316825, 0.020824438570145487, 0.08350025383140558, 0.6347182042191846]
[0.09670492762468053, 0.6645829247112878, 0.6192096802995823, 0.402028439021769, 0.0956702249154826]
[0.21939508173188904, 0.6378035407654421, 0.5900120813223357, 0.9475721197828657, 0.9362741702568245]
[0.5920906334231977, 0.8027222746910339, 0.7820831768023722, 0.3165252596417607, 0.8031765753318116]
[0.06857678860508387, 0.7325300505981869, 0.8462650999089738, 0.3612878414286451, 0.3906443438184044]
[0.3449633049543228, 0.09479300414958458, 0.4592302835936417, 0.271188367432212, 0.5673854431130068]
[0.050361760620463825, 0.24285216983043645, 0.9249131340531822, 0.5928481337375051,

In [11]:
size(df,1)

66

In [12]:
rand(1:66)

30

In [13]:
typeof(df.ln_n_months)

Array{Any,1}

In [14]:
function bootstrap_mm1(df, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(df,1)
    Bootstrap = zeros(B)
    
    
    
    for b in 1:B
        
        mm1 = Array{Any}(undef, S)
        
        for i in 1:S
            mm1[i] = df.ln_n_months[rand(1:S)]
        end
        
        Bootstrap[b] = mean(skipmissing(mm1))
        
    end
    
    
    return Bootstrap
    
end


bootstrap_mm1 (generic function with 1 method)

In [15]:
B = 100000

var(bootstrap_mm1(df, B))

0.014911987746230409

MM2: fraction reorganized given that the case went into court

In [16]:
mm2 = INCOURT
mm2 = size(filter(mm2 -> mm2.outcome_bargain .== "reorganized", mm2),1)/ size(mm2,1)

0.84375

In [17]:
function bootstrap_mm2(df, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(df,1)
    Bootstrap = zeros(B)
    
    
    for b in 1:B
        
        mm2 = Array{Any}(undef, S)
        
        for i in 1:S
            mm2[i] = df.outcome_bargain[rand(1:S)]
        end
        
        Bootstrap[b] = size(filter(mm2 -> mm2 .== "reorganized", mm2),1)/ size(mm2,1)
        
    end
    
    
    
    
    return Bootstrap
    
end

bootstrap_mm2 (generic function with 1 method)

In [18]:
var(bootstrap_mm2(INCOURT, B))

0.0020692321663206867

MM3: avg ln duration of court cases in months

In [19]:
mm3 = mean(log.(INCOURT.months_start_plan))

2.760695631787451

In [20]:
function bootstrap_mm3(INCOURT, B)


    S = size(INCOURT,1)
    Bootstrap = zeros(B)

    for b in 1:B

        mm3 = Array{Any}(undef, S)

        for i in 1:S
            mm3[i] = INCOURT.months_start_plan[rand(1:S)]
        end

        Bootstrap[b] = mean(log.(mm3))

    end

    return Bootstrap
end

bootstrap_mm3 (generic function with 1 method)

In [21]:
B = 100000

var(bootstrap_mm3(INCOURT, B))

0.0026162866222762377

MM4: Fraction of cases incourt

In [22]:
mm4 = mean(df.in_court)

0.9696969696969697

In [23]:
function bootstrap_mm4(df, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(df,1)
    Bootstrap = zeros(B)
    
    for b in 1:B
        
        mm4 = Array{Any}(undef, S)
        
        for i in 1:S
            mm4[i] = df.in_court[rand(1:S)]
        end
        
        Bootstrap[b] = mean(mm4)
        
    end
    
    return Bootstrap
    
end

bootstrap_mm4 (generic function with 1 method)

In [24]:
var(bootstrap_mm4(df, B))

0.00044657718349451605

MM5: avg recovery rate for senior given precourt REORGANIZATION

In [25]:
PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome_bargain .== "reorganized", PRECOURT);

In [26]:
PRECOURT_REORG.R_senior

2-element Array{Any,1}:
 0.805482736871508
 0.441695046670649

In [27]:
mm5 = mean(PRECOURT.R_senior)

0.6235888917710785

In [28]:
function bootstrap_mm5(PRECOURT_REORG, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(PRECOURT_REORG,1)
    Bootstrap = zeros(B)
    
    
    for b in 1:B
        
        mm5 = Array{Any}(undef, S)
        
        for i in 1:S
            mm5[i] = PRECOURT_REORG.R_senior[rand(1:S)]
        end
        
        Bootstrap[b] = mean(mm5)
        
    end
    
    
    
    
    return Bootstrap
    
end

bootstrap_mm5 (generic function with 1 method)

In [29]:
var(bootstrap_mm5(PRECOURT_REORG, B))

0.016592225735905203

MM6: avg recovery arte for junior given precourt REORGANIZATION

In [30]:
PRECOURT_REORG.R_junior

2-element Array{Any,1}:
 0.805482736871508
 0.308326296593457

In [31]:
mm6 = mean(PRECOURT_REORG.R_junior)

0.5569045167324824

In [32]:
function bootstrap_mm6(PRECOURT_REORG, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(PRECOURT_REORG,1)
    Bootstrap = zeros(B)
    
    for b in 1:B
        
        mm6 = Array{Any}(undef, S)
        
        for i in 1:S
            mm6[i] = PRECOURT_REORG.R_junior[rand(1:S)]
        end
        
        Bootstrap[b] = mean(mm6)
        
    end
    
    return Bootstrap
    
end

bootstrap_mm6 (generic function with 1 method)

In [33]:
var(bootstrap_mm6(PRECOURT_REORG, B))

0.03091502566019953

MM7: junior creditor’s average fraction gain, conditional on an in-court reorganization

In [34]:
INCOURT_REORG = filter(INCOURT -> INCOURT.outcome_bargain .== "reorganized", INCOURT);

In [35]:
INCOURT_REORG.payoff_j = INCOURT_REORG.R_junior .* INCOURT_REORG.D_junior;
INCOURT_REORG.payoff_s = INCOURT_REORG.R_senior .* INCOURT_REORG.D_senior;

In [36]:
mm7 = INCOURT_REORG
mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s ))

0.7733661464016862

In [37]:
function bootstrap_mm7(INCOURT_REORG, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(INCOURT_REORG,1)
    Bootstrap = zeros(B)
    
    for b in 1:B
        
        mm7 = Array{Any}(undef, S)
        
        for i in 1:S
            r = rand(1:S) #salvei o número em r para selecionar as 3 colunas na mesma linha
            mm7[i] = INCOURT_REORG.payoff_j[r] / (INCOURT_REORG.payoff_j[r] + INCOURT_REORG.payoff_s[r])
        end
        
        Bootstrap[b] = mean(mm7)
        
    end
    
    return Bootstrap
    
end

bootstrap_mm7 (generic function with 1 method)

In [38]:
var(bootstrap_mm7(INCOURT_REORG, B))

0.0009617928185223142

MM8: total recovery rate given incourt REORGANIZATION

In [39]:
mm8 = INCOURT_REORG
mm8 = mean((mm8.payoff_s .+ mm8.payoff_j) ./ (mm8.D_senior .+ mm8.D_junior))

0.4977287552407836

In [40]:
function bootstrap_mm8(INCOURT_REORG, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(INCOURT_REORG,1)
    Bootstrap = zeros(B)
    
    for b in 1:B
        
        mm8 = Array{Any}(undef, S)
        
        for i in 1:S
            r = rand(1:S)
            mm8[i] = (INCOURT_REORG.payoff_j[r] + INCOURT_REORG.payoff_s[r]) / (INCOURT_REORG.D_senior[r] + INCOURT_REORG.D_junior[r])
        end
        
        Bootstrap[b] = mean(mm8)
        
    end
    
    return Bootstrap
    
end

bootstrap_mm8 (generic function with 1 method)

In [41]:
var(bootstrap_mm8(INCOURT_REORG, B))

0.0006774004583424037

MM9: slope coefficient of a regression of the log total recovery rate given incourt reorganization on case duration

In [45]:
mm9 = INCOURT_REORG
x = log.((mm9.payoff_s .+ mm9.payoff_j) ./ (mm9.D_senior .+ mm9.D_junior))
X = [ones(size(x,1)) x]

54×2 Array{Float64,2}:
 1.0  -0.815239
 1.0  -0.208243
 1.0  -1.0012
 1.0  -0.625145
 1.0  -0.763717
 1.0  -0.817344
 1.0  -0.204669
 1.0  -1.57827
 1.0  -0.802925
 1.0  -0.319793
 1.0  -0.955516
 1.0  -0.868515
 1.0  -1.56602
 ⋮    
 1.0  -0.00498754
 1.0  -0.447523
 1.0  -1.06303
 1.0  -0.303923
 1.0  -1.19345
 1.0  -0.668951
 1.0  -0.516135
 1.0  -0.315944
 1.0  -0.727992
 1.0  -0.996418
 1.0  -1.04896
 1.0  -0.817633

In [48]:
y = log.(mm9.months_start_plan)

54-element Array{Float64,1}:
 2.987364023883473
 1.8718021769015913
 3.094068174274854
 2.233592221507094
 3.246490991901174
 2.841025883793008
 3.2970706724532564
 2.4567357728213066
 2.5827388990250326
 2.2228201245251826
 2.819392788437586
 2.4959564859745824
 3.104586678466073
 ⋮
 2.272125885509337
 2.244249515981082
 2.445241393395566
 2.6414354520202235
 2.80537854506277
 2.6366735382599846
 2.5468376075184933
 2.237157287671591
 2.2046046846338423
 2.954910279033736
 2.987364023883473
 3.8732821771117156

In [51]:
mm9 = (X' * X) \ (X' * y)
mm9 = mm9[2]

-0.09822867075174749

In [55]:
using LinearAlgebra

In [73]:
a = zeros(10)

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [79]:
function bootstrap_mm9(INCOURT_REORG, B)
    
    #S vai ser sempre o tamanho da base de dados
    S = size(INCOURT_REORG,1)
    Bootstrap = zeros(B)
    
    for b in 1:B
        
        x = Array{Float64}(undef, S)
        y = Array{Float64}(undef, S)
        
        x = zeros(S)
        y = zeros(S)
        
        
        for i in 1:S
            r = rand(1:S)
            x[i] = log(    (INCOURT_REORG.payoff_j[r] + INCOURT_REORG.payoff_s[r]) / (INCOURT_REORG.D_senior[r] + INCOURT_REORG.D_junior[r]) ) 
            y[i] = log(INCOURT_REORG.months_start_plan[r])
        end
        
        X = [ones(size(x,1)) x]
        
        mm9 = (X' * X) \ (X' * y)
        mm9 = mm9[2] 
        
        Bootstrap[b] = mm9
        
    end
    
    return Bootstrap
    
end

bootstrap_mm9 (generic function with 1 method)

In [81]:
bootstrap_mm9(INCOURT_REORG, B)

100000-element Array{Float64,1}:
  0.04520364351282747
  0.04423472561183468
  0.09774741349138183
 -0.3455180230209685
 -0.10214372666223082
  0.15170197470818744
  0.039018191021012005
 -0.0772734831224661
  0.09877702487504032
  0.022180889801752975
  0.002851272848129114
 -0.294689601215541
 -0.04253867179831926
  ⋮
 -0.2989201622510334
 -0.09902244356837321
  0.0975584126175234
  0.13560483357624023
 -0.05846310374552311
 -0.0852890744011216
  0.04742675447335224
 -0.2228474914368736
 -0.11006163707540317
 -0.06965414528870728
  0.005018140075281715
 -0.1652533470443733

Criando a matriz e invertendo

In [82]:
using LinearAlgebra
W_hat = Matrix{Int}(I, 9, 9);

In [83]:
B = 66 * 10000
w1 = var(bootstrap_mm1(df, B))
w2 = var(bootstrap_mm2(INCOURT, B))
w3 = var(bootstrap_mm3(INCOURT, B))
w4 = var(bootstrap_mm4(df, B))
w5 = var(bootstrap_mm5(PRECOURT_REORG, B))
w6 = var(bootstrap_mm6(PRECOURT_REORG, B))
w7 = var(bootstrap_mm7(INCOURT_REORG, B))
w8 = var(bootstrap_mm8(INCOURT_REORG, B))
w9 = var(bootstrap_mm9(INCOURT_REORG, B))

0.02005779282127588

In [85]:
weights = [w1, w2, w3, w4, w5, w6, w7, w8, w9]

9-element Array{Float64,1}:
 0.014898870803595794
 0.002056731961170494
 0.0026057041369018283
 0.0004462705318302864
 0.01654015758863799
 0.03092649081084349
 0.0009625950047741417
 0.0006792048122887158
 0.02005779282127588

In [86]:
omega = W_hat .* weights

9×9 Array{Float64,2}:
 0.0148989  0.0         0.0        0.0          0.0        0.0        0.0          0.0          0.0
 0.0        0.00205673  0.0        0.0          0.0        0.0        0.0          0.0          0.0
 0.0        0.0         0.0026057  0.0          0.0        0.0        0.0          0.0          0.0
 0.0        0.0         0.0        0.000446271  0.0        0.0        0.0          0.0          0.0
 0.0        0.0         0.0        0.0          0.0165402  0.0        0.0          0.0          0.0
 0.0        0.0         0.0        0.0          0.0        0.0309265  0.0          0.0          0.0
 0.0        0.0         0.0        0.0          0.0        0.0        0.000962595  0.0          0.0
 0.0        0.0         0.0        0.0          0.0        0.0        0.0          0.000679205  0.0
 0.0        0.0         0.0        0.0          0.0        0.0        0.0          0.0          0.0200578

In [87]:
inv(omega)

9×9 Array{Float64,2}:
 67.1192    0.0      0.0       0.0    0.0      0.0        0.0      0.0    0.0
  0.0     486.208    0.0       0.0    0.0      0.0        0.0      0.0    0.0
  0.0       0.0    383.773     0.0    0.0      0.0        0.0      0.0    0.0
  0.0       0.0      0.0    2240.79   0.0      0.0        0.0      0.0    0.0
  0.0       0.0      0.0       0.0   60.4589   0.0        0.0      0.0    0.0
  0.0       0.0      0.0       0.0    0.0     32.3347     0.0      0.0    0.0
  0.0       0.0      0.0       0.0    0.0      0.0     1038.86     0.0    0.0
  0.0       0.0      0.0       0.0    0.0      0.0        0.0   1472.31   0.0
  0.0       0.0      0.0       0.0    0.0      0.0        0.0      0.0   49.8559